In [1]:

import numpy as np
import argparse
# import tqdm
import pickle
import pandas as pd
import os
import datetime
import warnings
import seaborn as sns
from matplotlib import pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

from utility.utility import *
from utility.training import callback
import tensorflow as tf

import configparser

# turn off system warning 
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)


c:\Users\yayay\.virtualenvs\StockPricePrediction-MviMxm2U\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### take data from ml-lstm

In [2]:

data = pickle.load(open('data.pk', 'rb'))

c:\Users\yayay\.virtualenvs\StockPricePrediction-MviMxm2U\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator MinMaxScaler from version 1.0.2 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [3]:

data.stockPrice   
priceSet=data.stockPrice
display(priceSet.head(3)) 
# display(targetSet.iloc[:].head(1))

,0,1,2,3,4,5,6,7,8,9,...,34,35,36,37,38,39,40,41,42,43
0,-0.531974,-0.979487,-0.867862,-0.604503,-0.704443,-0.611049,-0.404988,-0.884426,0.705216,-0.647578,...,-0.195646,-0.979320,-0.882948,0.540395,-0.976658,-0.271402,0.044118,-0.879489,1.000000,-0.871450
1,-0.582389,-0.974358,-0.853058,-0.610290,-0.703443,-0.563007,-0.421337,-0.887295,0.716429,-0.609040,...,-0.232431,-0.980996,-0.874393,0.480043,-0.961096,-0.276113,-0.073529,-0.874491,0.966781,-0.872209
2,-0.597094,-0.993323,-0.885761,-0.634593,-0.709441,-0.688526,-0.533492,-0.885656,0.660312,-0.628830,...,-0.210360,-0.991057,-0.873289,0.200326,-0.943589,-0.290245,-0.078431,-0.884487,0.847192,-0.871450


In [4]:
# data.trainSet[0]['target_history']
data.validSet[0]['target_history']

array([[0.38968795],
       [0.38528013],
       [0.35001714],
       [0.38968795],
       [0.38528013],
       [0.40952358],
       [0.39409599],
       [0.41833921],
       [0.40291185],
       [0.38307622],
       [0.38307622],
       [0.36764863],
       [0.37426059],
       [0.38748404],
       [0.36764863],
       [0.38968795],
       [0.40070771],
       [0.39850381],
       [0.36985254],
       [0.36764863]])

### Test Process

In [6]:
import numpy as np
import argparse
import tqdm
import pickle
import pandas as pd
import os
import datetime
import warnings
import seaborn as sns
from matplotlib import pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

from utility.utility import *
from utility.training import callback
import tensorflow as tf

import configparser

# turn off system warning 
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

mode = 'train'
model_type='milstm'
config = configparser.ConfigParser()
config.read('config.ini')

start_time = datetime.datetime.now()

datetime_prefix = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# basic setting
make_dir(config)
maybe_make_dir(f'logs/{mode}/{model_type}')


# logging    
print(f'Training Start')

# reading data
print(f'Load Data Begin')
df = pickle.load(open('data.pk', 'rb'))
# df = read_data(config, args.mode)
if mode == 'train':
    train  = df.trainSet 
    valid  = df.validSet 
    if model_type == 'milstm':
        target_train=[]
        pos_train=[]
        neg_train=[]
        index_train=[]
        y_train=[]
        for i in range(0,len(train)):
            target_train.append(train[i]['target_history'])
            pos_train.append(train[i]['pos_history'])
            neg_train.append(train[i]['neg_history'])
            index_train.append(train[i]['index_history'])
            y_train.append(train[i]['target_price'])

        X_train = {
            'target_history':target_train,
            'pos_history':pos_train,
            'neg_history':neg_train,
            'index_history':index_train,
        }
        y_train = np.array(y_train) 

        target_valid=[]
        pos_valid=[]
        neg_valid=[]
        index_valid=[]
        y_valid=[]
        for i in range(0,len(valid)):
            target_valid.append(valid[i]['target_history'])
            pos_valid.append(valid[i]['pos_history'])
            neg_valid.append(valid[i]['neg_history'])
            index_valid.append(valid[i]['index_history'])
            y_valid.append(valid[i]['target_price'])

        X_valid = {
            'target_history':target_valid,
            'pos_history':pos_valid,
            'neg_history':neg_valid,
            'index_history':index_valid,
        }
        y_valid = np.array(y_valid)

        model = load_model(X_valid, model_type)

    else:    
        X_train=[]
        y_train=[]
        for i in range(0,len(train)):
            X_train.append(train[i]['target_history'])
            y_train.append(train[i]['target_price'])
        X_train = np.array(X_train)
        y_train = np.array(y_train)

        X_valid=[]
        y_valid=[]
        for i in range(0,len(valid)):
            X_valid.append(valid[i]['target_history'])
            y_valid.append(valid[i]['target_price'])
        X_valid = np.array(X_valid)
        y_valid = np.array(y_valid)

        model = load_model(X_valid.shape, model_type)
    
if mode == 'test':
    test  = df.testSet 
    X_test=[]
    y_test=[]
    for i in range(0,len(test)):
        X_test.append(test[i]['target_history'])
        y_test.append(test[i]['target_price'])
    X_test = np.array(X_test)
    y_test = np.array(y_test)

    model = load_model(X_test.shape, model_type)

print(f'Load Data Finish')

# print(args.mode) 
if mode == 'train':
    history = model.fit(
        X_train, y_train, 
        epochs=int(config['MODEL']['epoch']),
        verbose = 1,
        batch_size=512,
        validation_data=(X_valid, y_valid), 
        callbacks = callback(config, args, datetime_prefix)
    )
    y_pred = model.predict(X_valid)

    model.save_weights(f'mi_model/{model_type}/{model_type}')

    valid_mse = mean_squared_error(y_valid.reshape(-1,1), y_pred, squared=False)   
    print('mse',valid_mse)
    pd.DataFrame(history.history).to_csv(f'logs/csv_logger/{model_type}/{datetime_prefix}_{valid_mse}.csv')

if mode == 'test':
    model.load_weights(f'mi_model/{model_type}/{model_type}')
    y_pred = model.predict(X_test)
    # y_inverse = inverse_predict(y_pred, config)
    mse = mean_squared_error(y_test.reshape(-1,1), y_pred, squared=False)   
    
    print(model_type,'mse',mse)



Training Start
Load Data Begin


c:\Users\yayay\.virtualenvs\StockPricePrediction-MviMxm2U\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator MinMaxScaler from version 1.0.2 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


NameError: name 'n_obs' is not defined

In [12]:
target = np.array(X_valid['target_history']).shape
target[1]


20